# Equity of Travel Time Changes

In this workbook we analyze how changes in travel time and demand brought on by the introduciton of additional transit infrastructure is distributed accross populations.

First, we set up our modules and load the appropriate data.

In [1]:
import os
import pandas as pd
import altair as alt
import numpy as np

# Replace with your data folder paths
mx_folder = r"data/matrices/input"
counts_folder = r"data/counts/input"
link_folder = r"data/counts/interim"
output_folder = r"data/counts/output"

In [2]:
dtypes = {'i':int, 'j':int, 'travel_time':float, 'demand':float}
mx_BAU = pd.read_csv(os.path.join(mx_folder, 'bau_times_flows.csv'), dtype=dtypes)
mx_A = pd.read_csv(os.path.join(mx_folder, 'scenario_A_times_flows.csv'), dtype=dtypes)
mx_B = pd.read_csv(os.path.join(mx_folder, 'scenario_B_times_flows.csv'), dtype=dtypes)
mx_C = pd.read_csv(os.path.join(mx_folder, 'scenario_C_times_flows.csv'), dtype=dtypes)

In [7]:
mx_BAU['mult'] = mx_BAU['travel_time'] * mx_BAU['demand']
wtt_BAU = mx_BAU[['i', 'travel_time', 'demand', 'mult']].groupby('i', as_index=False).sum()
wtt_BAU['wtt_BAU'] = wtt_BAU['mult']/wtt_BAU['demand']
wtt_BAU

mx_A['mult'] = mx_A['travel_time'] * mx_A['demand']
wtt_A = mx_A[['i', 'travel_time', 'demand', 'mult']].groupby('i', as_index=False).sum()
wtt_A['wtt_A'] = wtt_A['mult']/wtt_A['demand']

mx_B['mult'] = mx_B['travel_time'] * mx_B['demand']
wtt_B = mx_B[['i', 'travel_time', 'demand', 'mult']].groupby('i', as_index=False).sum()
wtt_B['wtt_B'] = wtt_B['mult']/wtt_B['demand']

mx_C['mult'] = mx_C['travel_time'] * mx_C['demand']
wtt_C = mx_C[['i', 'travel_time', 'demand', 'mult']].groupby('i', as_index=False).sum()
wtt_C['wtt_C'] = wtt_C['mult']/wtt_C['demand']

delta = pd.merge(wtt_BAU[['i', 'wtt_BAU']], wtt_A[['i', 'wtt_A']], on='i')
delta = pd.merge(delta, wtt_B[['i', 'wtt_B']], on='i')
delta = pd.merge(delta, wtt_C[['i', 'wtt_C']], on='i')
delta['delta_A'] = delta['wtt_BAU'] - delta['wtt_A']
delta['delta_B'] = delta['wtt_BAU'] - delta['wtt_B']
delta['delta_C'] = delta['wtt_BAU'] - delta['wtt_C']
delta.to_csv(os.path.join(output_folder, 'travel_time_change.csv'), index=False)

In [8]:
taz_da = pd.read_csv(os.path.join(link_folder, 'taz_da_link.csv'), dtype={'DAUID': int, 'taz_id': int, 'frac_da_in_taz': float})
da_demo = pd.read_csv(os.path.join(counts_folder, 'da_census_profile.csv'))
delta_demo = pd.merge(delta, taz_da, left_on='i', right_on='taz_id')
delta_demo = pd.merge(delta_demo, da_demo, on='DAUID')
delta_da = delta_demo[delta_demo.columns[-11:-1]].multiply(delta_demo['frac_da_in_taz'], axis="index")
delta_da.columns = [f"f_{c}" for c in delta_da.columns]
delta_demo = pd.concat([delta_demo[['delta_A', 'delta_B', 'delta_C']], delta_da], axis=1)

# Now we do the weighted summary
delta_demo['A_pop_2016'] = (delta_demo['f_pop_2016']/delta_demo['f_pop_2016'].sum())*delta_demo['delta_A']
delta_demo['A_vm_minority'] = (delta_demo['f_vm_minority']/delta_demo['f_vm_minority'].sum())*delta_demo['delta_A']
delta_demo['A_income_lim'] = (delta_demo['f_income_lim']/delta_demo['f_income_lim'].sum())*delta_demo['delta_A']
delta_demo['A_labour_unemployed'] = (delta_demo['f_labour_unemployed']/delta_demo['f_labour_unemployed'].sum())*delta_demo['delta_A']

delta_demo['B_pop_2016'] = (delta_demo['f_pop_2016']/delta_demo['f_pop_2016'].sum())*delta_demo['delta_B']
delta_demo['B_vm_minority'] = (delta_demo['f_vm_minority']/delta_demo['f_vm_minority'].sum())*delta_demo['delta_B']
delta_demo['B_income_lim'] = (delta_demo['f_income_lim']/delta_demo['f_income_lim'].sum())*delta_demo['delta_B']
delta_demo['B_labour_unemployed'] = (delta_demo['f_labour_unemployed']/delta_demo['f_labour_unemployed'].sum())*delta_demo['delta_B']

delta_demo['C_pop_2016'] = (delta_demo['f_pop_2016']/delta_demo['f_pop_2016'].sum())*delta_demo['delta_C']
delta_demo['C_vm_minority'] = (delta_demo['f_vm_minority']/delta_demo['f_vm_minority'].sum())*delta_demo['delta_C']
delta_demo['C_income_lim'] = (delta_demo['f_income_lim']/delta_demo['f_income_lim'].sum())*delta_demo['delta_C']
delta_demo['C_labour_unemployed'] = (delta_demo['f_labour_unemployed']/delta_demo['f_labour_unemployed'].sum())*delta_demo['delta_C']


In [9]:
to_plot = delta_demo[[
    'A_pop_2016', 'B_pop_2016', 'C_pop_2016', 
    'A_vm_minority', 'B_vm_minority', 'C_vm_minority', 
    'A_income_lim', 'B_income_lim', 'C_income_lim',
    'A_labour_unemployed', 'B_labour_unemployed', 'C_labour_unemployed'
    ]]

pretty_names = {'income_lim': "Low Income (LIM)", 'pop_2016': 'Total Population', 'vm_minority': "Visible Minority", 'labour_unemployed':"Unemployed"}
melted = to_plot.melt().groupby('variable', as_index=False).sum()
melted['scenario'] = melted.variable.str[0]
melted['demographic'] = melted.variable.str[2:]
melted['demo_name'] = melted.demographic.map(pretty_names)
melted

,variable,value,scenario,demographic,demo_name
0,A_income_lim,0.991225,A,income_lim,Low Income (LIM)
1,A_labour_unemployed,0.991737,A,labour_unemployed,Unemployed
2,A_pop_2016,1.001199,A,pop_2016,Total Population
3,A_vm_minority,1.071866,A,vm_minority,Visible Minority
4,B_income_lim,0.322596,B,income_lim,Low Income (LIM)
5,B_labour_unemployed,0.315265,B,labour_unemployed,Unemployed
6,B_pop_2016,0.300060,B,pop_2016,Total Population
7,B_vm_minority,0.403077,B,vm_minority,Visible Minority
8,C_income_lim,0.026967,C,income_lim,Low Income (LIM)
9,C_labour_unemployed,0.004714,C,labour_unemployed,Unemployed


In [11]:
alt.Chart(melted).mark_bar().encode(
    alt.Y('scenario:N', title=None),
    alt.X('value:Q', title='Average Travel Time Savings (min)'),
    alt.Color('scenario:N', title='Scenario'),
    alt.Row('demo_name:N', title='', sort=['Total Population'], spacing=35)
).properties(
    title="Savings in Average Travel Time for SmartTrack Scenarios",
    width=600,
    height=80
).configure(font='Roboto').configure_axis(grid=False).configure_view(strokeWidth=0).configure_title(fontSize=18)

alt.Chart(...)